# OpenAI Assistant with Autogen and Postgres

This example hows how you can use the OpenAI Assistant agents with autogen, to solve database related tasks (Postgres DB).

In [1]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

from modules import llm
from modules import rand
from turbo4 import Turbo4
from modules import embeddings
from modules.instruments import PostgresAgentInstruments

import autogen
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent


DB_URL = os.environ.get("POSTGRES_CONNECTION_URL")
POSTGRES_TABLE_DEFINITIONS_CAP_REF = "TABLE_DEFINITIONS"

In [ ]:
# ! pip install --upgrade pyautogen

In [2]:
raw_prompt = "Get the mean age of those who have had covid."
prompt = f"Fulfill this database query: {raw_prompt}. "

assistant_name = "test"
session_id = rand.generate_session_id(assistant_name + raw_prompt)
assistant_id = Turbo4().get_or_create_assistant(assistant_name).assistant_id

get_or_create_assistant(test, gpt-4-1106-preview)


In [3]:
# Define the SQL tool schema
sql_tool_schema = {
    "name": "run_sql_tool",
    "parameters": {
        "type": "object",
        "properties": {
        "query": {
            "type": "string",
            "description": "SQL query to be executed"
        }
        },
        "required": ["query"]
    },
    "description": "Executes SQL queries and returns results."
    }

# Configuration for GPTAssistantAgent
config_list = autogen.config_list_from_dotenv('../../')
llm_config = {
    "config_list": config_list,
    "assistant_id": assistant_id,
    "tools": [sql_tool_schema]
}

# Initialize the assistant
sql_assistant = GPTAssistantAgent(
    name="SQL_Assistant",
    instructions=autogen.AssistantAgent.DEFAULT_SYSTEM_MESSAGE,
    llm_config=llm_config
)

# Initialize User Proxy Agent
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    code_execution_config={
        "work_dir": "./autogen_results",
        "use_docker": False
    },
    human_input_mode="NEVER",
)

In [4]:
with PostgresAgentInstruments(DB_URL, session_id) as (agent_instruments, db):
    
    # Register the run_sql function
    sql_assistant.register_function(
        function_map={
            "run_sql_tool": agent_instruments.run_sql
        }
    )

    # Get table definitions from Postgres
    database_embedder = embeddings.DatabaseEmbedder(db)
    table_definitions = database_embedder.get_similar_table_defs_for_prompt(
        raw_prompt
    )

    # Forumalte prompt
    prompt = llm.add_cap_ref(
        prompt,
        f"Use these {POSTGRES_TABLE_DEFINITIONS_CAP_REF} to satisfy the database query.",
        POSTGRES_TABLE_DEFINITIONS_CAP_REF,
        table_definitions,
    )

    # Initiate autogen chat
    user_proxy.initiate_chat(sql_assistant, message=prompt)


user_proxy (to SQL_Assistant):

Fulfill this database query: Get the mean age of those who have had covid..  Use these TABLE_DEFINITIONS to satisfy the database query.

TABLE_DEFINITIONS

CREATE TABLE patients (
id integer,
patient_id character varying(255),
state character varying(2),
fips integer,
diagnosed_covid boolean,
diagnoses_date date,
current_age integer,
age_at_diagnosis double precision
);

CREATE TABLE patient_medical_history (
id integer,
patient_id character varying(255),
has_diabetes boolean,
has_heart_disease boolean,
has_lung_disease boolean,
smoker_status character varying(10),
bmi double precision,
last_checkup_date date
);

--------------------------------------------------------------------------------
SQL_Assistant (to user_proxy):

To fulfill the database query, I will write a SQL query that calculates the mean age of patients who have been diagnosed with COVID-19. Since we are interested in the current age of the patients, we'll use the `current_age` column fro

KeyboardInterrupt: 